In [12]:
import numpy as np
import pickle
# torch
import torch

# quimb
import quimb.tensor as qtn
import symmray as sr
from autoray import do
from vmc_torch.hamiltonian_torch import spinful_Fermi_Hubbard_square_lattice_torch


# Hamiltonian parameters
Lx = int(3)
Ly = int(2)
symmetry = 'Z2'
t = 1.0
U = 8.0
N_f = int(Lx*Ly-2)
n_fermions_per_spin = (N_f//2, N_f//2)
H = spinful_Fermi_Hubbard_square_lattice_torch(Lx, Ly, t, U, N_f, pbc=False, n_fermions_per_spin=n_fermions_per_spin)
graph = H.graph
# TN parameters
D = 4
chi = -1
dtype=torch.float64

# Load PEPS
pwd = '.'
skeleton = pickle.load(open(pwd+f'/peps_skeleton_{Lx}x{Ly}_{symmetry}.pkl', "rb"))
peps_params = pickle.load(open(pwd+f'/peps_su_params_{Lx}x{Ly}_{symmetry}.pkl', "rb"))
peps = qtn.unpack(peps_params, skeleton)
for ts in peps.tensors:
    ts.data.phase_sync(inplace=True)
## 2. Scale the tensor elements
scale = 4.0
peps.apply_to_arrays(lambda x: torch.tensor(scale*x, dtype=dtype))
## 3. Set the exponent to 0.0
peps.exponent = 0.0

# Tensors' oddpos
for ts in peps.tensors:
    print(ts.data.oddpos)

()
()
()
()
()
()


In [13]:
from symmray.fermionic_local_operators import FermionicOperator
class fPEPS(qtn.PEPS):
    def __init__(self, arrays, *, shape="urdlp", tags=None, site_ind_id="k{},{}", site_tag_id="I{},{}", x_tag_id="X{}", y_tag_id="Y{}", **tn_opts):
        super().__init__(arrays, shape=shape, tags=tags, site_ind_id=site_ind_id, site_tag_id=site_tag_id, x_tag_id=x_tag_id, y_tag_id=y_tag_id, **tn_opts)
        self.symmetry = self.arrays[0].symmetry
        self.spinless = True if self.phys_dim() == 2 else False
    
    def product_bra_state(self, config, reverse=1):
        product_tn = qtn.TensorNetwork()
        backend = self.tensors[0].data.backend
        dtype = eval(backend+'.'+self.tensors[0].data.dtype)
        if isinstance(config, np.ndarray):
            kwargs = {'like':config, 'dtype':dtype}
        elif isinstance(config, torch.Tensor):
            device = list(self.tensors[0].data.blocks.values())[0].device
            kwargs = {'like':config, 'device':device, 'dtype':dtype}
        if self.spinless:
            index_map = {0: 0, 1: 1}
            array_map = {
                0: do('array',[1.0,],**kwargs), 
                1: do('array',[1.0,],**kwargs)
            }
        else:
            if self.symmetry == 'Z2':
                index_map = {0:0, 1:1, 2:1, 3:0}
                array_map = {
                    0: do('array',[1.0, 0.0],**kwargs), 
                    1: do('array',[1.0, 0.0],**kwargs), 
                    2: do('array',[0.0, 1.0],**kwargs), 
                    3: do('array',[0.0, 1.0],**kwargs)
                }
            elif self.symmetry == 'U1':
                index_map = {0:0, 1:1, 2:1, 3:2}
                array_map = {
                    0: do('array',[1.0,],**kwargs), 
                    1: do('array',[1.0, 0.0],**kwargs), 
                    2: do('array',[0.0, 1.0],**kwargs), 
                    3: do('array',[1.0,],**kwargs)
                }
            elif self.symmetry == 'U1U1':
                index_map = {0:(0,0), 1:(0,1), 2:(1,0), 3:(1,1)}
                array_map = {
                    0: do('array',[1.0],**kwargs),
                    1: do('array',[1.0],**kwargs), 
                    2: do('array',[1.0],**kwargs),
                    3: do('array',[1.0],**kwargs)
                }

        for n, site in zip(config, self.sites):
            p_ind = self.site_ind_id.format(*site)
            p_tag = self.site_tag_id.format(*site)
            tid = self.sites.index(site)

            n_charge = index_map[int(n)]
            n_array = array_map[int(n)]

            oddpos = None
            if not self.spinless:
                # assert self.symmetry == 'U1', "Only U1 symmetry is supported for spinful fermions for now."
                if int(n) == 1:
                    oddpos = (3*tid+1)*(-1)**reverse
                elif int(n) == 2:
                    oddpos = (3*tid+2)*(-1)**reverse
                elif int(n) == 3:
                    # oddpos = ((3*tid+1)*(-1)**reverse, (3*tid+2)*(-1)**reverse)
                    oddpos = None
            else:
                oddpos = (3*tid+1)*(-1)**reverse

            tsr_data = sr.FermionicArray.from_blocks(
                blocks={(n_charge,):n_array}, 
                duals=(True,),
                symmetry=self.symmetry, 
                charge=n_charge, 
                oddpos=oddpos
            )
            tsr = qtn.Tensor(data=tsr_data, inds=(p_ind,),tags=(p_tag, 'bra'))
            product_tn |= tsr

        return product_tn
    
    
    def fix_phys_inds(self, sites, config, inplace=False):
        """Slicing to get the amplitude locally, faster than contraction with a tensor product state.
        Note here sites is a list of tuples (x, y) for the sites to be fixed, and config is a 1D array of integers representing the fermion occupation numbers at those sites."""
        peps = self if inplace else self.copy()
        backend = self.tensors[0].data.backend
        dtype = eval(backend+'.'+self.tensors[0].data.dtype)
        if isinstance(config, np.ndarray): 
            kwargs = {'like':config, 'dtype':dtype}
        elif isinstance(config, torch.Tensor):
            device = list(self.tensors[0].data.blocks.values())[0].device
            kwargs = {'like':config, 'device':device, 'dtype':dtype}
        if self.spinless:
            raise NotImplementedError("Efficient amplitude calculation is not implemented for spinless fermions.")
        else:
            if self.symmetry == 'Z2':
                index_map = {0:0, 1:1, 2:1, 3:0}
                array_map = {
                    0: do('array',[1.0, 0.0],**kwargs), 
                    1: do('array',[1.0, 0.0],**kwargs), 
                    2: do('array',[0.0, 1.0],**kwargs), 
                    3: do('array',[0.0, 1.0],**kwargs)
                }
            elif self.symmetry == 'U1':
                index_map = {0:0, 1:1, 2:1, 3:2}
                array_map = {
                    0: do('array',[1.0,],**kwargs), 
                    1: do('array',[1.0, 0.0],**kwargs), 
                    2: do('array',[0.0, 1.0],**kwargs), 
                    3: do('array',[1.0,],**kwargs)
                }

            for n, site in zip(config, sites):
                p_ind = peps.site_ind_id.format(*site)
                tid = peps.sites.index(site)
                fts = peps.tensor_map[tid]
                ftsdata = fts.data
                ftsdata.phase_sync(inplace=True) # explicitly apply all lazy phases that are stored and not yet applied
                phys_ind_order = fts.inds.index(p_ind)
                charge = index_map[int(n)]
                input_vec = array_map[int(n)]
                charge_sec_data_dict = ftsdata.blocks
                new_fts_inds = fts.inds[:phys_ind_order] + fts.inds[phys_ind_order+1:]
                new_charge_sec_data_dict = {}
                for charge_blk, data in charge_sec_data_dict.items():
                    if charge_blk[phys_ind_order] == charge:
                        new_data = do('tensordot', data, input_vec, axes=([phys_ind_order], [0]))
                        new_charge_blk = charge_blk[:phys_ind_order] + charge_blk[phys_ind_order+1:]
                        new_charge_sec_data_dict[new_charge_blk]=new_data
                        
                new_duals = ftsdata.duals[:phys_ind_order] + ftsdata.duals[phys_ind_order+1:]

                if int(n) == 1:
                    new_oddpos = (3*tid+1)*(-1)
                elif int(n) == 2:
                    new_oddpos = (3*tid+2)*(-1)
                elif int(n) == 3 or int(n) == 0:
                    new_oddpos = ()

                new_oddpos1 = FermionicOperator(new_oddpos, dual=True) if new_oddpos is not () else ()
                new_oddpos = ftsdata.oddpos + (new_oddpos1,) if new_oddpos1 is not () else ftsdata.oddpos
                oddpos = list(new_oddpos)[::-1]
                
                new_fts_data = sr.FermionicArray.from_blocks(new_charge_sec_data_dict, duals=new_duals, charge=charge+ftsdata.charge, oddpos=oddpos, symmetry=self.symmetry)
                fts.modify(data=new_fts_data, inds=new_fts_inds, left_inds=None)

            amp = qtn.PEPS(peps)

        return amp
    
    def get_amp(self, config, inplace=False, conj=True, reverse=1, contract=True, efficient=True, functional=False):
        """Get the amplitude of a configuration in a PEPS."""
        if functional:
            return self.get_amp_functional(config, inplace=inplace)
        if efficient:
            return self.get_amp_efficient(config, inplace=inplace)
        peps = self if inplace else self.copy()
        product_state = self.product_bra_state(config, reverse=reverse).conj() if conj else self.product_bra_state(config, reverse=reverse)
        
        amp = peps|product_state # ---T---<---|n>

        if not contract:
            return amp
        
        for site in peps.sites:
            site_tag = peps.site_tag_id.format(*site)
            amp.contract_(tags=site_tag)

        amp.view_as_(
            qtn.PEPS,
            site_ind_id="k{},{}",
            site_tag_id="I{},{}",
            x_tag_id="X{}",
            y_tag_id="Y{}",
            Lx=peps.Lx,
            Ly=peps.Ly,
        )
        return amp
    
    def get_amp_efficient(self, config, inplace=False):
        """Slicing to get the amplitude, faster than contraction with a tensor product state."""
        peps = self if inplace else self.copy()
        backend = self.tensors[0].data.backend
        dtype = eval(backend + '.' + self.tensors[0].data.dtype)
        if isinstance(config, np.ndarray):
            kwargs = {'like': config, 'dtype': dtype}
        elif isinstance(config, torch.Tensor):
            device = list(self.tensors[0].data.blocks.values())[0].device
            kwargs = {'like': config, 'device': device, 'dtype': dtype}
        
        if self.spinless:
            raise NotImplementedError("Efficient amplitude calculation is not implemented for spinless fermions.")
        else:
            if self.symmetry == 'Z2':
                index_map = {0: 0, 1: 1, 2: 1, 3: 0}
                array_map = {
                    0: do('array', [1.0, 0.0], **kwargs),
                    1: do('array', [1.0, 0.0], **kwargs),
                    2: do('array', [0.0, 1.0], **kwargs),
                    3: do('array', [0.0, 1.0], **kwargs)
                }
            elif self.symmetry == 'U1':
                index_map = {0: 0, 1: 1, 2: 1, 3: 2}
                array_map = {
                    0: do('array', [1.0], **kwargs),
                    1: do('array', [1.0, 0.0], **kwargs),
                    2: do('array', [0.0, 1.0], **kwargs),
                    3: do('array', [1.0], **kwargs)
                }
            elif self.symmetry == 'U1U1':
                index_map = {0:(0,0), 1:(0,1), 2:(1,0), 3:(1,1)}
                array_map = {
                    0: do('array',[1.0],**kwargs),
                    1: do('array',[1.0],**kwargs), 
                    2: do('array',[1.0],**kwargs),
                    3: do('array',[1.0],**kwargs)
                }
            else:
                raise ValueError(f"symmetry: {self.symmetry} , name: {self.symmetry.__class__.__name__}, type: {type(self.symmetry)}, array: {self.arrays[0].symmetry=='Z2'}")


            for n, site in zip(config, self.sites):
                p_ind = peps.site_ind_id.format(*site)
                site_id = peps.sites.index(site)
                site_tag = peps.site_tag_id.format(*site)
                # fts = peps.tensors[site_id]
                fts = peps[site_tag]
                ftsdata = fts.data # this is the on-site fermionic tensor (f-tensor) to be contracted
                ftsdata.phase_sync(inplace=True) # explicitly apply all lazy phases that are stored and not yet applied
                phys_ind_order = fts.inds.index(p_ind)
                charge = index_map[int(n)] # charge of the on-site fermion configuration
                input_vec = array_map[int(n)] # input vector of the on-site fermion configuration
                charge_sec_data_dict = ftsdata.blocks # the dictionary of the f-tensor data

                new_fts_inds = fts.inds[:phys_ind_order] + fts.inds[phys_ind_order + 1:] # calculate indices of the contracted f-tensor
                new_charge_sec_data_dict = {} # new dictionary to store the data of the contracted f-tensor
                for charge_blk, data in charge_sec_data_dict.items():
                    if charge_blk[phys_ind_order] == charge:
                        # 1. Determine which index to select (0 or 1) from the input vector.
                        #    `argmax` finds the position of the '1.0'
                        select_index = torch.argmax(input_vec).item()

                        # 2. Build the slicer tuple dynamically.
                        #    This creates a list of `slice(None)` (which is equivalent to `:`)
                        #    and inserts the `select_index` at the correct position.
                        slicer = [slice(None)] * data.ndim
                        slicer[phys_ind_order] = select_index

                        # 3. Apply the slice to get the new data.
                        new_data = data[tuple(slicer)]

                        new_charge_blk = charge_blk[:phys_ind_order] + charge_blk[phys_ind_order + 1:] # new charge block
                        new_charge_sec_data_dict[new_charge_blk] = new_data # new charge block and its corresponding data in a dictionary

                new_duals = ftsdata.duals[:phys_ind_order] + ftsdata.duals[phys_ind_order + 1:]

                if int(n) == 1:
                    new_oddpos = (3 * site_id + 1) * (-1)
                elif int(n) == 2:
                    new_oddpos = (3 * site_id + 2) * (-1)
                elif int(n) == 3 or int(n) == 0:
                    new_oddpos = ()

                new_oddpos1 = FermionicOperator(new_oddpos, dual=True) if new_oddpos is not () else ()
                new_oddpos = ftsdata.oddpos + (new_oddpos1,) if new_oddpos1 is not () else ftsdata.oddpos
                oddpos = list(new_oddpos)[::-1]
                try:
                    if self.symmetry == 'U1':
                        new_charge = charge + ftsdata.charge
                    elif self.symmetry == 'Z2':
                        new_charge = (charge + ftsdata.charge) % 2 # Z2 symmetry, charge should be 0 or 1
                    elif self.symmetry == 'U1U1':
                        new_charge = (charge[0] + ftsdata.charge[0], charge[1] + ftsdata.charge[1]) # U1U1 symmetry, charge should be a tuple of two integers
                    new_fts_data = sr.FermionicArray.from_blocks(new_charge_sec_data_dict, duals=new_duals, charge=new_charge, oddpos=oddpos, symmetry=self.symmetry)
                except Exception:
                    # Error when constructing the new f-tensor
                    print(n, site, phys_ind_order, charge_sec_data_dict, new_charge_sec_data_dict)
                    
                fts.modify(data=new_fts_data, inds=new_fts_inds, left_inds=None)

            amp = qtn.PEPS(peps)

            return amp

In [14]:
from vmc_torch.fermion_utils import from_netket_config_to_quimb_config
all_states_quimb = np.array([from_netket_config_to_quimb_config(s) for s in H.hilbert.all_states()])
H_dense = np.zeros((len(all_states_quimb), len(all_states_quimb)), dtype=np.float64)
quimb_state_index_mapping = {tuple(s): i for i, s in enumerate(all_states_quimb)}
for i, s1 in enumerate(all_states_quimb):
    connected_states, coeff = H.get_conn(s1)
    for s2, c in zip(connected_states, coeff):
        j = quimb_state_index_mapping[tuple(s2)]
        H_dense[j, i] = c
H_dense = torch.tensor(H_dense) # Hubbard Hamiltonian projected to fixed N (U1 charge) and Sz=0 sector

In [15]:
psi_vec = []
peps.apply_to_arrays(lambda x: torch.tensor(x, dtype=dtype) if not isinstance(x, torch.Tensor) else x)
for config in all_states_quimb:
    amp = peps.get_amp_efficient(torch.tensor(config)).contract()
    psi_vec.append(amp)
psi_vec = np.array(psi_vec)

E_psi = np.conj(psi_vec) @ H_dense.numpy() @ psi_vec / (np.conj(psi_vec) @ psi_vec) / (Lx*Ly)
E_psi # Energy of psi projected to fixed N (U1 charge) and Sz=0 sector
print(f'VMC energy per site: {E_psi}')

VMC energy per site: -0.6863537947574306


In [16]:
import quimb.tensor as qtn
import symmray as sr

# SU in quimb
edges = qtn.edges_2d_square(Lx, Ly, cyclic=False)
parse_edges_to_site_info = sr.parse_edges_to_site_info
site_info = parse_edges_to_site_info(
    edges,
    4,
    phys_dim=4,
    site_ind_id="k{},{}",
    site_tag_id="I{},{}",
)

t = 1.0
U = 8.0

terms = {
    (sitea, siteb): sr.fermi_hubbard_local_array(
        t=t, U=U, mu=0.0,
        symmetry=symmetry,
        coordinations=(
            site_info[sitea]['coordination'],
            site_info[siteb]['coordination'],
        ),
    )
    for (sitea, siteb) in peps.gen_bond_coos()
}
ham = qtn.LocalHam2D(Lx, Ly, terms)

peps.apply_to_arrays(lambda x: x.numpy() if isinstance(x, torch.Tensor) else x)
print('Sanity check:')
print(f'Quimb energy per site: {peps.compute_local_expectation_exact(ham.terms)/ (Lx*Ly)}')

Sanity check:
Quimb energy per site: -0.6961658795654624
